<a href="https://colab.research.google.com/github/ekanshi258/optimization-algos/blob/master/clustered_design_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymoo

This experiment is based on the following paper:  
_Solving structural engineering design optimization problems using an artificial bee colony algorithm_: https://www.aimsciences.org/article/doi/10.3934/jimo.2014.10.777

I will be using the NSGA2 to solve the same problem as given in the paper.


In [2]:
import numpy as np
from pymoo.model.problem import FunctionalProblem
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation, get_termination
from pymoo.optimize import minimize
from sklearn.datasets import make_blobs

The paper states 3 structural design problems. 

Problem 3:  
**Tension/Compression String Design. (Section 4.2.3).**


---


Objective functions:  
```
Minimize f(X) = (x3 + 2) * x2 * x1^2
Such that:
g1(X) = 1 − (x2^3 * x3)/(71785 * x1^4) <= 0
g2(X) = (4*x2^2 − x1*x2)/12566*(x2*x1^3 − x1^4) + 1/(5108 * x1^2) − 1 <= 0
g3(X) = 1 − (140.45 * x1)/(x2^2 * x3) <= 0
g4(X) = (x1 + x2)/(1.5) − 1 <= 0

Bounds:
0.05 <= x1 <= 2 ; 0.25 <= x2 <= 1.3 ; 2 <= x3 <= 15
```

> Minimizing the weight of a tension/compression spring subject to constraints on minimum deflection, shear stress, surge frequency, limits on outside diameter and on design
variables. The design variables are the mean coil diameter(x1), the wire diameter (x2) and the number of active coil (x3).

In [3]:
#objective functions
objs = [
    lambda x: (x[2] + 2) * x[1] * x[0]**2,
]

# Contraint equations
constr_ieq = [
    lambda x: 1 - (x[1]**3 * x[2])/(71785 * x[0]**4),
    lambda x: (4*x[1]**2 - x[0]*x[1])/(12566*(x[1] * x[0]**3 - x[0]**4)) + 1/(5108 * x[0]**2) - 1,
    lambda x: 1 - (140.45 * x[0])/(x[1]**2 * x[2]),
    lambda x: (x[0] + x[1])/(1.5) - 1
]

# value bounds
lower_bound = np.array([0.05, 0.25, 2])
upper_bound = np.array([2, 1.3, 15])

# initialising problem
problem = FunctionalProblem(3, objs, constr_ieq=constr_ieq, xl=lower_bound, xu=upper_bound)


Instead of starting out with a randomly generated population as we usually do, I will be generating clusters of populations, i.e. the population will be clustered into groups instead of being scattered throughout the solution space.

**Reason**: Comparison of algo/solution quality with that when population is randomly scattered.

In [4]:
# Generating 30 clusters:
X, y = make_blobs(n_samples=300, centers=30, n_features=3, random_state=0)

In [5]:
algorithm = NSGA2(
    pop_size= 100,     
    n_offsprings=25,
    sampling = X,
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 280)

Optimizing:

In [ ]:
ans = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

In [7]:
print("X: ", ans.X)
print("Obj: ", ans.F)
print("Constraints: ", ans.G)

X:  [0.06954457 0.94403834 2.00000014]
Obj:  [0.01826317]
Constraints:  [0. 0. 0. 0.]


In [8]:
ans.exec_time

2.9024665355682373

**Observation**

The solution obtained from clustering is much worse than that obtained without clustering (`0.01317647`) as well as that given in the paper using ABC (`0.01266`). 

Tuning of the population size, number of clusters, etc. may give better results. 